# Employment DataBase

In [3]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.filterwarnings("ignore")

In [4]:
def connect_to_db():
    connection = psycopg2.connect(user = 'postgres' , password = 1212,
                                host = 'localhost',
                                port = 5432,
                                database = 'employee_db')
    return connection
connection = connect_to_db()    

### Checking all the tables

In [5]:
department_name = pd.read_sql_query("select * from employees.department",connection)
department_name

,id,dept_name
0,d001,Marketing
1,d002,Finance
2,d003,Human Resources
3,d004,Production
4,d005,Development
5,d006,Quality Management
6,d007,Sales
7,d008,Research
8,d009,Customer Service


In [6]:
department_employee = pd.read_sql_query("select * from employees.department_employee",connection)
department_employee.head()

,employee_id,department_id,from_date,to_date
0,10001,d005,1986-06-26,9999-01-01
1,10002,d007,1996-08-03,9999-01-01
2,10003,d004,1995-12-03,9999-01-01
3,10004,d004,1986-12-01,9999-01-01
4,10005,d003,1989-09-12,9999-01-01


In [7]:
department_manager =  pd.read_sql_query("select * from employees.department_manager",connection)
department_manager.head()

,employee_id,department_id,from_date,to_date
0,110022,d001,1985-01-01,1991-10-01
1,110039,d001,1991-10-01,9999-01-01
2,110085,d002,1985-01-01,1989-12-17
3,110114,d002,1989-12-17,9999-01-01
4,110183,d003,1985-01-01,1992-03-21


In [8]:
employee =  pd.read_sql_query("select * from employees.employee",connection)
employee.head()

,id,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [9]:
salary = pd.read_sql_query("select * from employees.salary",connection)
salary.head()

,employee_id,amount,from_date,to_date
0,10001,60117,1986-06-26,1987-06-26
1,10001,62102,1987-06-26,1988-06-25
2,10001,66074,1988-06-25,1989-06-25
3,10001,66596,1989-06-25,1990-06-25
4,10001,66961,1990-06-25,1991-06-25


In [10]:
title = pd.read_sql_query("select * from employees.title",connection)
title.head()

,employee_id,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


### Find counts of each and every table

In [11]:
print(department_name.shape)
print(department_employee.shape)
print(department_manager.shape)
print(employee.shape)
print(salary.shape)
print(title.shape)

(9, 2)
(331603, 4)
(24, 4)
(300024, 6)
(1141783, 4)
(443308, 4)


###  Which department has the highest average salary of active employees ?

In [24]:
def get_department_wise_avg_salary():
    query = """ select dept_name ,avg(amount)as avg_salary
                from employees.department_employee de
                LEFT JOIN employees.salary  s
                ON s.employee_id = de.employee_id
                LEFT JOIN employees.department d
                ON d.id = de.department_id
                where extract(year from s.to_date) = 9999 AND extract(year from de.to_date) = 9999
                group by dept_name 
                order by avg_salary desc"""

    connection = connect_to_db()
    salary_info_all_dept = pd.read_sql_query(query, connection) 
    connection.close()  

    plt.figure(figsize=(7,5))
    sns.barplot(data = salary_info_all_dept, x = "dept_name", y = "avg_salary")
    plt.xlabel("Department")
    plt.ylabel("Average Salary")
    plt.title("Department-wise Average Salary")
    plt.xticks(rotation=90)  
    plt.tight_layout()  
    plt.show()
    return salary_info_all_dept[salary_info_all_dept['avg_salary'] == max(salary_info_all_dept['avg_salary'])]


 Sales Department has the highest average salary then followed by Marketing and Finance

### Which title has the highest avg salary?

In [32]:
def get_title_wise_avg_salary():
    query = """ select title ,avg(amount) AS avg_salary
            from employees.title t
            inner join employees.salary  s
            ON t.employee_id = s.employee_id
            where extract(year from s.to_date) = 9999 and extract(year from t.to_date) = 9999
            group by title 
            order by avg_salary desc"""

    connection = connect_to_db()
    salary_info_all_titles = pd.read_sql_query(query,connection)
    connection.close() 
    
    sns.barplot(data = salary_info_all_titles,x = "title",y = "avg_salary")
    plt.xlabel("Department")
    plt.ylabel("Average Salary")
    plt.title("Title-wise Average Salary")
    plt.xticks(rotation=90)  
    plt.tight_layout()  
    plt.show()
    return  salary_info_all_titles[salary_info_all_titles['avg_salary'] == max(salary_info_all_titles['avg_salary'])]

Senior Staff has the highest average salary followed by staff then manager

### Distribution of salary across titles.

In [41]:
def salary_dist_across_titles():
    query = """ select title, sum(amount) as amount 
            from employees.title t
            inner join employees.salary s
            on t.employee_id = s.employee_id
            where extract(year from s.to_date) = 9999 and extract(year from t.to_date) = 9999
            group by title """

    connection = connect_to_db()
    distribution_of_salary_of_titles = pd.read_sql_query(query, connection)
    connection.close()  

    plt.pie(distribution_of_salary_of_titles['amount'], labels=distribution_of_salary_of_titles['title'], autopct='%1.1f%%')
    plt.title("Titlewise Salary Distribution")
    plt.axis('equal')  
    plt.tight_layout()
    plt.show()

### Distribution of salary across departments

In [44]:
def salary_dist_across_depts():
    query = """ select dept_name, sum(amount) as amount
            from employees.department_employee de
            inner join employees.salary s
            on de.employee_id= s.employee_id
            inner join employees.department d
            on d.id = de.department_id
            where extract(year from de.to_date) = 9999 and extract(year from s.to_date) = 9999
            group by dept_name """

    connection = connect_to_db()
    salary_of_departments = pd.read_sql_query(query,connection)
    connection.close()  

    plt.pie(salary_of_departments['amount'], labels=salary_of_departments['dept_name'], autopct='%1.1f%%')
    plt.title("Department-wise Salary Distribution")
    plt.axis('equal') 
    plt.tight_layout()
    plt.show()

###  How many active managers in each department. Is there any department with no manager?


In [53]:
def active_manager_in_dept():
    query = """ select dept_name, count(employee_id) as count 
            from employees.department_manager dm
            left join employees.department d
            on dm.department_id = d.id
            where extract(year from dm.to_date) = 9999
            group by dept_name """

    connection = connect_to_db()
    active_managers = pd.read_sql_query(query, connection)
    connection.close()  
    
    sns.countplot(data = active_managers, x = "dept_name")
    plt.xlabel("Department")
    plt.ylabel("count of managers")
    plt.title("Count of managers")
    plt.xticks(rotation=90)  
    plt.tight_layout()  
    plt.show()

For each department there is an active manager

### Composition of titles department-wise

In [63]:
def title_composition_dept_wise():
    query = """ SELECT d.dept_name as dept_name, t.title as title, count(e.id) as count
            FROM employees.employee e
            LEFT JOIN employees.title t
            ON t.employee_id = e.id
            LEFT JOIN employees.department_employee de
            ON de.employee_id = e.id
            LEFT JOIN employees.department d
            ON d.id = de.department_id
            where extract(year from t.to_date) = 9999 and extract(year from de.to_date) = 9999
            GROUP BY d.dept_name, t.title """

    connection = connect_to_db()
    title_dept_wise = pd.read_sql_query(query, connection)
    connection.close()  

    plt.figure(figsize=(10, 5))
    sns.barplot(x='dept_name',y = "count", hue='title', data=title_dept_wise)
    plt.title('Distribution of Titles within Departments')
    plt.xlabel('Department')
    plt.ylabel('Title_count')
    plt.xticks(rotation=90)
    plt.legend(title='Titles', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

### Composition of departments title-wise.

In [67]:
def dept_composition_title_wise():
        query = """ SELECT t.title as title, d.dept_name as dept_name 
                FROM employees.employee e
                LEFT JOIN employees.title t
                ON t.employee_id = e.id
                LEFT JOIN employees.department_employee de
                ON de.employee_id = e.id
                LEFT JOIN employees.department d
                ON d.id = de.department_id 
                where extract(year from t.to_date) = 9999 """

        connection = connect_to_db()
        dept_title_wise = pd.read_sql_query(query, connection)
        connection.close()  

        plt.figure(figsize=(10, 5))
        sns.countplot(x='title', hue='dept_name', data=dept_title_wise)
        plt.title('Distribution of Departments within Titles')
        plt.xlabel('Titles')
        plt.ylabel('Count')
        plt.xticks(rotation=90)
        plt.legend(title='Departments', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.grid()
        plt.show()

### Salaries of active department managers Which department's manager who is active earns the most?

In [71]:
def dept_manager_salaries():
    query = """ SELECT dept_name , sum(amount) as amount
                from employees.department_manager dm
                inner join employees.salary s 
                on dm.employee_id = s.employee_id
                inner join employees.department d
                on d.id = dm.department_id
                where extract(year from dm.to_date) = 9999 and extract(year from s.to_date) = 9999
                group by dept_name
                order by amount desc """

    connection = connect_to_db()
    active_dept_manager_salary = pd.read_sql_query(query, connection)
    connection.close()  

    sns.barplot(data = active_dept_manager_salary, x = "dept_name", y = "amount")
    plt.xlabel("Department")
    plt.ylabel("Salary")
    plt.title("Salaries of active department managers")
    plt.xticks(rotation=90)  
    plt.tight_layout()  
    plt.show()
    return active_dept_manager_salary[active_dept_manager_salary['amount']==max(active_dept_manager_salary['amount'])]

In the marketing department the manager earns the most 

### What are the titles of active department managers? 

In [81]:
def get_active_manager_titles():
       connection = connect_to_db()
       query = """ select distinct(t.title) from employees.department_manager dm 
                left join employees.title t 
                on dm.employee_id = t.employee_id 
				where date_part('year', dm.to_date) =9999 
			    and date_part('year', t.to_date) = 9999 """
       active_manager_title = pd.read_sql_query(query, connection)
       connection.close()
       return active_manager_title['title'].tolist()


### Past history of salaries of managers across department (yearly)

In [87]:
def get_manager_past_salaries_dept_wise():
    connection = connect_to_db()
    query = """SELECT dm.employee_id, d.dept_name, s.amount, s.from_date, s.to_date FROM employees.department_manager dm join employees.salary s 
              on dm.employee_id = s.employee_id join employees.department d
			  on d.id = dm.department_id
			  where s.from_date >= dm.from_date
			  and s.to_date <= dm.to_date"""
    df= pd.read_sql_query(query, connection)
    df['labels'] = df['from_date'].apply(str) + " to " + df['to_date'].apply(str)
    depts= df['dept_name'].unique()
    employees_manager = df['employee_id'].unique()
    for i in depts:
        for j in employees_manager:
            new_df= df[(df['dept_name']==i) & (df['employee_id']==j)]
            if not new_df.empty:
                my_plot= sns.barplot(x= new_df['labels'], y= new_df['amount'])
                for item in my_plot.get_xticklabels():
                    item.set_rotation(90)
                plt.title(i+str(j))
                plt.show()
            else:
                continue

Marketing department has highest salary for manager 

### Distribution of salaries employees working for more than 10 years vs 4 years vs 1 year.

In [89]:
def exp_mapper(x):
    if x>=10:
        return '10+ years'
    elif x >=4:
        return '4 to 10 years'
    elif x >=1:
        return '1 to 4 years'
    else:
        return 'New Joiners'
    

def get_salary_dist_exp_wise():
    connection = connect_to_db()
    
    query = """ select s.amount, date_part('year',de.to_date)- date_part('year',de.from_date) as exp 
			from employees.department_employee de join  employees.salary s 
			on s.employee_id = de.employee_id
			where date_part('year',de.to_date)- date_part('year',de.from_date) <= 60
			and date_part('year',s.to_date) = 9999;"""
                        
    df= pd.read_sql_query(query, connection)
    df['exp'] = df['exp'].astype('int')
    
    ### Aux function to be kept outside ideally
    
    df['exp_status'] = df['exp'].apply(lambda x: exp_mapper(x))
    sns.violinplot(x= df['exp_status'], y = df['amount'])

### Average number of years employees work in the company before leaving (title wise)

In [92]:
def avg_years_in_company_title_wise():
    query = """SELECT t.title,
        AVG(EXTRACT(YEAR FROM t.to_date) - EXTRACT(YEAR FROM e.hire_date)) AS avg_years_before_leaving
        FROM  employees.employee e
        LEFT JOIN employees.title t 
        ON t.employee_id = e.id
        WHERE date_part('year', t.to_date) != 9999
        GROUP BY t.title """
    connection = connect_to_db()
    employees_work_year = pd.read_sql_query(query,connection)
    connection.close()  

    sns.pointplot(data =employees_work_year, x='title', y='avg_years_before_leaving', color='blue')
    plt.xlabel("Title")
    plt.ylabel("avg_years_before_leaving")
    plt.title("Average years before leaving (Title-wise)")
    plt.xticks(rotation=90) 
    plt.tight_layout()  
    plt.show()

### Average number of years employees work in the company before leaving (Dept wise)

In [97]:
def avg_years_in_company_department_wise():
    query = """SELECT d.dept_name,AVG(EXTRACT(YEAR FROM t.to_date) - EXTRACT(YEAR FROM e.hire_date)) AS avg_years_before_leaving
            FROM employees.employee e
            LEFT JOIN employees.title t
            ON t.employee_id = e.id
            LEFT JOIN employees.department_employee de
            ON de.employee_id = e.id
            LEFT JOIN employees.department d 
            ON d.id = de.department_id
            WHERE date_part('year',t.to_date) != 9999 and extract(year from de.to_date) != 9999 
            GROUP BY d.dept_name;
        
                    """

    connection = connect_to_db()
    employees_work_year = pd.read_sql_query(query,connection)
    connection.close() 

    sns.pointplot(data =employees_work_year, x='dept_name', y='avg_years_before_leaving', color='blue')
    plt.xlabel("dept_name")
    plt.ylabel("avg_years_before_leaving")
    plt.title("Average years before leaving (Dept-wise)")
    plt.xticks(rotation=90)  
    plt.tight_layout()  
    plt.show()

### Median annual salary increment department wise

In [102]:
def get_median_salry_inc_dept_wise():
    query = """SELECT dept_name, s.amount as salary
            FROM employees.department_employee de
            left JOIN employees.salary s
            ON de.employee_id = s.employee_id
            left JOIN employees.department d
            ON de.department_id = d.id """

    connection = connect_to_db()
    median_annual_salary = pd.read_sql_query(query,connection)
    connection.close()  

    median_salary = median_annual_salary.groupby("dept_name")["salary"].median()
    median_salary.plot(kind = "bar",color='skyblue')
    plt.title('Median Salary by Department')
    plt.xlabel('Departments')
    plt.ylabel('Median Salary')
    plt.xticks(rotation=45, ha='right')  
    plt.tight_layout()
    plt.show()

Sales has the highest median salary increment then followed by marketing and finance